In [15]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import cv2
import tensorflow as tf

In [5]:
# Step 1: Segmentation using k-means clustering
def segment_image(image_path, num_clusters):
    image = np.array(Image.open(image_path))
    height, width, _ = image.shape
    reshaped_image = image.reshape(-1, 3)

    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans.fit(reshaped_image)
    segmented_image = kmeans.labels_.reshape(height, width)
    return segmented_image

In [11]:
# Step 2: Object detection using YOLO (You Only Look Once)
def detect_objects(image_path):
    net = cv2.dnn.readNet("yolov8m.pt")
    classes = []
    with open("coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Post-processing for object detection
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    return boxes, confidences, class_ids

In [12]:
# Step 3: Visualize segmentation and detected objects
def visualize_segmentation_and_objects(image_path, segmented_image, boxes):
    image = cv2.imread(image_path)
    for box in boxes:
        x, y, w, h = box
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(segmented_image, cmap='jet')
    plt.title('Segmentation')
    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title('Object Detection')
    plt.show()

In [13]:
# Example usage
image_path = 'images/3.jpg'
segmented_image = segment_image(image_path, num_clusters=5)
segmented_image

array([[4, 4, 4, ..., 3, 3, 3],
       [4, 4, 4, ..., 4, 4, 4],
       [4, 4, 4, ..., 4, 4, 4],
       ...,
       [3, 3, 3, ..., 4, 4, 4],
       [3, 3, 3, ..., 4, 4, 4],
       [3, 3, 3, ..., 4, 4, 4]])

In [14]:
boxes, _, _ = detect_objects(image_path)
visualize_segmentation_and_objects(image_path, segmented_image, boxes)

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\dnn_read.cpp:58: error: (-2:Unspecified error) Cannot determine an origin framework of files: yolov8m.pt in function 'cv::dnn::dnn4_v20231225::readNet'
